In this notebook, we will convert the dicom files into nifti format. Also, we will use the SRI24 dataset to resample the images to a consistent orientation. This is, within the same modality - say T2w, the sequene for different patient appear in different orientation (axial, sagittal etc). We will use the SRI24 axial image as the referece image to resample our data.

The following notebooks/tutorial were of great help.

1. https://www.kaggle.com/boojum/connecting-voxel-spaces/
2. https://simpleitk.readthedocs.io/en/master/link_examples.html

# Imports and paths

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
from tqdm import tqdm

import nibabel as nib
import SimpleITK as sitk

from fastai.medical.imaging import *
from fastai.vision.all import *

In [ ]:
path_train =  Path('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train')

In [ ]:
path_train_t2w,path_train_t1wce,path_train_t1w,path_train_flair = [],[],[],[]
for each in path_train.ls():
    path_train_t2w.append(each.ls()[0])
    path_train_t1wce.append(each.ls()[1])
    path_train_t1w.append(each.ls()[2])
    path_train_flair.append(each.ls()[3])

# Functions

In [ ]:
def get_array(fn):
    "opens .nii file and return the array"
    img = sitk.ReadImage(str(fn))
    imgd = sitk.GetArrayFromImage(img)
    return imgd

def plot_slice(imgd, sli):
    "given an image of shape slices x height x width, plots a slice"
    plt.imshow(imgd[sli], cmap='gray')
    plt.axis('off')

In [ ]:
def dicom2nifti(image_dir, save=True):
    "given a dicom directory, loads them into single file and can save it as .nii file"
    reader = sitk.ImageSeriesReader()
    reader.LoadPrivateTagsOn()
    filenamesDICOM = reader.GetGDCMSeriesFileNames(str(image_dir))
    reader.SetFileNames(filenamesDICOM)
    img = reader.Execute()
    img = sitk.Cast(img, sitk.sitkFloat32)
    
    if save:
        sitk.WriteImage(img, f'/kaggle/working/T2w/{image_dir.parent.name}.nii')
    else:
        return img

In [ ]:
def resample_nifti(image_dir, ref_image, fn, save=True):
    "resample using a reference image"

    image = dicom2nifti(image_dir, save=False)
    
    initial_transform = sitk.CenteredTransformInitializer(ref_image, 
                                                          image, 
                                                          sitk.Euler3DTransform(), 
                                                          sitk.CenteredTransformInitializerFilter.GEOMETRY)

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ref_image)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetTransform(initial_transform)
    resampler.SetOutputSpacing(ref_image.GetSpacing())
    resampler.SetSize((ref_image.GetSize()))
    resampler.SetOutputDirection(ref_image.GetDirection())
    resampler.SetOutputOrigin(ref_image.GetOrigin())
    resampler.SetDefaultPixelValue(image.GetPixelIDValue())
    resamped_image = resampler.Execute(image)
    
    if save:
        sitk.WriteImage(resamped_image, fn)

    return resamped_image

In [ ]:
#code from simpleitk examples
def threshold_based_crop_and_bg_median(image):
    '''
    Use Otsu's threshold estimator to separate background and foreground. In medical imaging the background is
    usually air. Then crop the image using the foreground's axis aligned bounding box and compute the background 
    median intensity.
    Args:
        image (SimpleITK image): An image where the anatomy and background intensities form a bi-modal distribution
                                 (the assumption underlying Otsu's method.)
    Return:
        Cropped image based on foreground's axis aligned bounding box.
        Background median intensity value.
    '''
    # Set pixels that are in [min_intensity,otsu_threshold] to inside_value, values above otsu_threshold are
    # set to outside_value. The anatomy has higher intensity values than the background, so it is outside.
    inside_value = 0
    outside_value = 255
    bin_image = sitk.OtsuThreshold(image, inside_value, outside_value)

    # Get the median background intensity
    label_intensity_stats_filter = sitk.LabelIntensityStatisticsImageFilter()
    label_intensity_stats_filter.SetBackgroundValue(outside_value)
    label_intensity_stats_filter.Execute(bin_image,image)
    bg_median = label_intensity_stats_filter.GetMedian(inside_value)
    
    # Get the bounding box of the anatomy
    label_shape_filter = sitk.LabelShapeStatisticsImageFilter()    
    label_shape_filter.Execute(bin_image)
    bounding_box = label_shape_filter.GetBoundingBox(outside_value)
    # The bounding box's first "dim" entries are the starting index and last "dim" entries the size
    return bg_median, sitk.RegionOfInterest(image, bounding_box[int(len(bounding_box)/2):], bounding_box[0:int(len(bounding_box)/2)])

# Let's see what we are talking about 

Let's open one and see how it looks like

In [ ]:
samp = path_train_t2w[0]

In [ ]:
samp_img = dicom2nifti(samp, False)
samp_imgd = sitk.GetArrayFromImage(samp_img)

In [ ]:
samp_imgd.shape

In [ ]:
plot_slice(samp_imgd, 100)

# Now lets resample using a SIR24 as referece image

In [ ]:
ref_image = sitk.ReadImage("../input/sri24-dataset/sri24/late.nii", sitk.sitkFloat32)

In [ ]:
samp_resamp = resample_nifti(samp, ref_image, "", False)
samp_resampd = sitk.GetArrayFromImage(samp_resamp)

We have reshaped the samp images to the same dimensions as the reference image

In [ ]:
sitk.GetArrayFromImage(ref_image).shape, samp_resampd.shape

And we have resampled to the axial view! 

In [ ]:
plot_slice(samp_resampd, 98)

We can also crop them to the region of interest

In [ ]:
_,samp_resamp_cropped = threshold_based_crop_and_bg_median(samp_resamp)
samp_resamp_croppedd = sitk.GetArrayFromImage(samp_resamp_cropped)

In [ ]:
samp_resamp_croppedd.shape

In [ ]:
plot_slice(samp_resamp_croppedd, 80)

As you can see the first few slices have no region of interest. By cropping to the ROI, we remove the not useful slices. 

In [ ]:
plot_slice(samp_resampd, 0)

In [ ]:
plot_slice(samp_resamp_croppedd, 0)

# Resample all T2w images

In [ ]:
!mkdir t2w_preproc_v2

In [ ]:
ref_image = sitk.ReadImage('../input/sri24-dataset/sri24/spgr.nii', sitk.sitkFloat32)

In [ ]:
for fn in tqdm(path_train_t2w, total=len(path_train_t2w)):
    pat_id = str(fn).split('/')[-2]
    final_fn = f"/kaggle/working/t2w_preproc_v2/{pat_id}.nii.gz"
    resample_nifti(fn, ref_image, final_fn, True)


In [ ]:
from zipfile import ZipFile
import os
import shutil


# iterate over all the files in directory
for folderName, subfolders, filenames in os.walk(f'/kaggle/working/t2w_preproc_v2'):
    # create a ZipFile object
    #print(folderName)
    with ZipFile(folderName.split('/')[-1] + '.zip', 'w') as zipObj:
        for filename in filenames:
            # create complete filepath of file in directory
            filePath = os.path.join(folderName, filename)
            # add file to zip
            zipObj.write(filePath, os.path.basename(filePath))
            # delete the file to open space
            os.remove(filePath)
shutil.rmtree(f"/kaggle/working/t2w_preproc_v2")
